In [ ]:
# MEDIS Pharmaceutical Sales Forecasting Analysis
## ATOR Product Line - Competitive Intelligence & Forecasting

**Objective**: Create the best forecasting model for MEDIS laboratory sales, taking into account:
- Historical sales data from recent years
- Competitor performance by sub-market
- Seasonal patterns and market dynamics
- Hierarchical product structure (dosage categories, package sizes)

**Dataset**: 7+ years of monthly pharmaceutical sales data (April 2018 - April 2025)
- **Main Product**: ATOR (cholesterol medication)
- **Competitors**: 15+ laboratories in the market
- **Market Segments**: 4 dosage categories (10mg, 20mg, 40mg, 80mg)
- **Package Sizes**: Primarily 30 and 90 tablet packages


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


In [ ]:
# Load the pharmaceutical sales data
print("=== LOADING MEDIS PHARMACEUTICAL SALES DATA ===")

# Load main dataset
df = pd.read_excel('MEDIS_VENTES.xlsx', sheet_name='Data')

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['ANNEE_MOIS'].min()} to {df['ANNEE_MOIS'].max()}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display column information
print("\n=== COLUMN DESCRIPTIONS ===")
column_descriptions = {
    'laboratoire': 'Pharmaceutical manufacturer (MEDIS + competitors)',
    'PRODUIT': 'Commercial product name',  
    'PRESENTATION': 'Complete product description (dosage, form, package)',
    'SOUS_MARCHE': 'Sub-market grouping products with same dosage',
    'PACK': 'Number of tablets per box',
    'ANNEE_MOIS': 'Reference month (YYYYMM format)',
    'VENTE_IMS': 'Monthly boxes sold (IMS estimates from pharmacies)',
    'qte': 'Additional quantity measure',
    'QTE_STOCK_A': 'Stock quantity (annual)',
    'QTE_STOCK_Q': 'Stock quantity (quarterly)', 
    'NBJ_RUPTURE': 'Number of stock-out days'
}

for col, desc in column_descriptions.items():
    if col in df.columns:
        print(f"• {col:15}: {desc}")

print(f"\nFirst few rows:")
df.head()


In [ ]:
# Competitive Landscape Analysis
print("=== COMPETITIVE LANDSCAPE ANALYSIS ===")

# Laboratory (competitor) analysis
print("\n1. COMPETITORS BY MARKET PRESENCE:")
lab_counts = df['laboratoire'].value_counts()
print(lab_counts)

# MEDIS market position
medis_data = df[df['laboratoire'] == 'MEDIS']
print(f"\n2. MEDIS POSITION:")
print(f"• MEDIS has {len(medis_data):,} records out of {len(df):,} total ({len(medis_data)/len(df)*100:.1f}%)")
print(f"• MEDIS products: {medis_data['PRODUIT'].unique()}")

# Sub-market analysis
print(f"\n3. SUB-MARKETS (by dosage):")
submarket_counts = df['SOUS_MARCHE'].value_counts()
print(submarket_counts)

# Package size distribution
print(f"\n4. PACKAGE SIZES:")
pack_counts = df['PACK'].value_counts()
print(pack_counts)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Competitor market presence
lab_counts.head(10).plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Top 10 Laboratories by Record Count')
axes[0,0].set_xlabel('Laboratory')
axes[0,0].set_ylabel('Number of Records')
axes[0,0].tick_params(axis='x', rotation=45)

# Sub-market distribution
submarket_counts.plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%')
axes[0,1].set_title('Market Share by Dosage Category')

# Package size distribution
pack_counts.plot(kind='bar', ax=axes[1,0], color='lightcoral')
axes[1,0].set_title('Package Size Distribution')
axes[1,0].set_xlabel('Package Size (tablets)')
axes[1,0].set_ylabel('Number of Records')

# MEDIS vs Competitors
medis_vs_others = pd.Series({
    'MEDIS': len(medis_data),
    'Competitors': len(df) - len(medis_data)
})
medis_vs_others.plot(kind='pie', ax=axes[1,1], autopct='%1.1f%%', colors=['green', 'orange'])
axes[1,1].set_title('MEDIS vs Competitors (Records)')

plt.tight_layout()
plt.show()


In [ ]:
# Data Preprocessing for Time Series Analysis
print("=== DATA PREPROCESSING ===")

# Convert ANNEE_MOIS to proper datetime
df['date'] = pd.to_datetime(df['ANNEE_MOIS'].astype(str), format='%Y%m')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

# Clean and standardize package sizes (round to 30/60/90 as suggested)
print("\nOriginal package sizes:", sorted(df['PACK'].dropna().unique()))

def standardize_pack_size(pack):
    if pd.isna(pack):
        return pack
    if pack <= 35:
        return 30
    elif pack <= 70:
        return 60
    else:
        return 90

df['pack_std'] = df['PACK'].apply(standardize_pack_size)
print("Standardized package sizes:", sorted(df['pack_std'].dropna().unique()))

# Focus on main sales metric
df['sales'] = df['VENTE_IMS'].fillna(0)

# Create hierarchical structure for MEDIS data
print(f"\n=== MEDIS DATA STRUCTURE ===")
medis_df = df[df['laboratoire'] == 'MEDIS'].copy()

print(f"MEDIS records: {len(medis_df):,}")
print(f"Date range: {medis_df['date'].min()} to {medis_df['date'].max()}")
print(f"Products: {medis_df['PRODUIT'].unique()}")
print(f"Sub-markets: {medis_df['SOUS_MARCHE'].unique()}")

# Check for missing data
print(f"\n=== DATA QUALITY CHECK ===")
print("Missing values per column:")
missing_data = medis_df.isnull().sum()
print(missing_data[missing_data > 0])

# Summary statistics for MEDIS sales
print(f"\n=== MEDIS SALES SUMMARY ===")
medis_sales_summary = medis_df.groupby(['SOUS_MARCHE', 'pack_std'])['sales'].agg([
    'count', 'sum', 'mean', 'std', 'min', 'max'
]).round(2)
print(medis_sales_summary)


In [ ]:
# Time Series Analysis of MEDIS Sales
print("=== MEDIS SALES TIME SERIES ANALYSIS ===")

# Aggregate MEDIS sales by month and sub-market
medis_monthly = medis_df.groupby(['date', 'SOUS_MARCHE'])['sales'].sum().reset_index()
medis_monthly_pivot = medis_monthly.pivot(index='date', columns='SOUS_MARCHE', values='sales').fillna(0)

# Calculate total monthly sales
medis_monthly_pivot['TOTAL'] = medis_monthly_pivot.sum(axis=1)

print("Monthly sales data shape:", medis_monthly_pivot.shape)
print("Date range:", medis_monthly_pivot.index.min(), "to", medis_monthly_pivot.index.max())

# Create comprehensive time series visualizations
fig, axes = plt.subplots(3, 2, figsize=(18, 15))

# 1. Total MEDIS sales over time
medis_monthly_pivot['TOTAL'].plot(ax=axes[0,0], linewidth=2, color='darkblue')
axes[0,0].set_title('MEDIS Total Monthly Sales', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Sales (boxes)')
axes[0,0].grid(True, alpha=0.3)

# 2. Sales by sub-market
for col in medis_monthly_pivot.columns[:-1]:  # Exclude TOTAL
    medis_monthly_pivot[col].plot(ax=axes[0,1], label=col, linewidth=2)
axes[0,1].set_title('MEDIS Sales by Sub-Market', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Sales (boxes)')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Year-over-year comparison
yearly_sales = medis_monthly_pivot['TOTAL'].resample('Y').sum()
yearly_sales.plot(kind='bar', ax=axes[1,0], color='green', alpha=0.7)
axes[1,0].set_title('MEDIS Annual Sales', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('Sales (boxes)')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Seasonal patterns
monthly_avg = medis_monthly_pivot['TOTAL'].groupby(medis_monthly_pivot.index.month).mean()
monthly_avg.plot(kind='bar', ax=axes[1,1], color='orange', alpha=0.7)
axes[1,1].set_title('MEDIS Average Monthly Sales (Seasonal Pattern)', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Month')
axes[1,1].set_ylabel('Average Sales (boxes)')
axes[1,1].set_xticks(range(12))
axes[1,1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# 5. Growth rates
growth_rates = medis_monthly_pivot['TOTAL'].pct_change(12) * 100  # Year-over-year growth
growth_rates.plot(ax=axes[2,0], color='red', linewidth=2)
axes[2,0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2,0].set_title('MEDIS Year-over-Year Growth Rate', fontsize=14, fontweight='bold')
axes[2,0].set_ylabel('Growth Rate (%)')
axes[2,0].grid(True, alpha=0.3)

# 6. Market share evolution (placeholder for now)
# We'll calculate this when we analyze competitive dynamics
axes[2,1].text(0.5, 0.5, 'Market Share Analysis\n(To be calculated with\ncompetitive data)', 
               ha='center', va='center', transform=axes[2,1].transAxes, fontsize=12)
axes[2,1].set_title('MEDIS Market Share Evolution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print key insights
print(f"\n=== KEY INSIGHTS ===")
print(f"• Total MEDIS sales in dataset: {medis_monthly_pivot['TOTAL'].sum():,.0f} boxes")
print(f"• Average monthly sales: {medis_monthly_pivot['TOTAL'].mean():,.0f} boxes")
print(f"• Peak month: {medis_monthly_pivot['TOTAL'].idxmax()} ({medis_monthly_pivot['TOTAL'].max():,.0f} boxes)")
print(f"• Lowest month: {medis_monthly_pivot['TOTAL'].idxmin()} ({medis_monthly_pivot['TOTAL'].min():,.0f} boxes)")
print(f"• Sales volatility (std): {medis_monthly_pivot['TOTAL'].std():,.0f} boxes")
print(f"• Latest 12-month growth: {growth_rates.iloc[-1]:.1f}%")


In [ ]:
# Competitive Analysis by Sub-Market
print("=== COMPETITIVE DYNAMICS ANALYSIS ===")

# Analyze competitive dynamics for each sub-market
submarkets = df['SOUS_MARCHE'].unique()

# Create competitive time series for each sub-market
competitive_data = {}
market_share_data = {}

for submarket in submarkets:
    submarket_df = df[df['SOUS_MARCHE'] == submarket]
    
    # Monthly sales by laboratory
    monthly_comp = submarket_df.groupby(['date', 'laboratoire'])['sales'].sum().reset_index()
    monthly_comp_pivot = monthly_comp.pivot(index='date', columns='laboratoire', values='sales').fillna(0)
    
    # Calculate market shares
    monthly_comp_pivot_pct = monthly_comp_pivot.div(monthly_comp_pivot.sum(axis=1), axis=0) * 100
    
    competitive_data[submarket] = monthly_comp_pivot
    market_share_data[submarket] = monthly_comp_pivot_pct
    
    print(f"\n{submarket} - Top competitors:")
    total_sales = monthly_comp_pivot.sum().sort_values(ascending=False)
    print(total_sales.head())

# Visualize competitive dynamics for the most important sub-markets
fig, axes = plt.subplots(2, 2, figsize=(20, 12))
axes = axes.flatten()

for i, submarket in enumerate(submarkets):
    if i >= 4:  # Only show first 4 sub-markets
        break
        
    # Get top 5 competitors in this sub-market
    comp_data = competitive_data[submarket]
    top_competitors = comp_data.sum().nlargest(5).index
    
    # Plot sales evolution
    for competitor in top_competitors:
        if competitor in comp_data.columns:
            comp_data[competitor].plot(ax=axes[i], label=competitor, linewidth=2)
    
    axes[i].set_title(f'{submarket} - Sales Evolution (Top 5 Competitors)', fontweight='bold')
    axes[i].set_ylabel('Monthly Sales (boxes)')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# MEDIS market share analysis
print(f"\n=== MEDIS MARKET SHARE ANALYSIS ===")

medis_market_shares = {}
for submarket in submarkets:
    share_data = market_share_data[submarket]
    if 'MEDIS' in share_data.columns:
        medis_shares = share_data['MEDIS']
        medis_market_shares[submarket] = {
            'current_share': medis_shares.iloc[-6:].mean(),  # Last 6 months average
            'peak_share': medis_shares.max(),
            'min_share': medis_shares.min(),
            'volatility': medis_shares.std()
        }
        print(f"\n{submarket}:")
        print(f"  • Current market share (6-month avg): {medis_shares.iloc[-6:].mean():.1f}%")
        print(f"  • Peak market share: {medis_shares.max():.1f}%")
        print(f"  • Market share volatility: {medis_shares.std():.1f}%")

# Create market share evolution chart
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
axes = axes.flatten()

for i, submarket in enumerate(submarkets):
    if i >= 4:
        break
    
    share_data = market_share_data[submarket]
    if 'MEDIS' in share_data.columns:
        share_data['MEDIS'].plot(ax=axes[i], linewidth=3, color='green', label='MEDIS')
        
        # Add top 2 competitors for context
        top_competitors = competitive_data[submarket].sum().nlargest(3).index
        top_competitors = [comp for comp in top_competitors if comp != 'MEDIS'][:2]
        
        for competitor in top_competitors:
            if competitor in share_data.columns:
                share_data[competitor].plot(ax=axes[i], linewidth=2, alpha=0.7, label=competitor)
    
    axes[i].set_title(f'{submarket} - Market Share Evolution', fontweight='bold')
    axes[i].set_ylabel('Market Share (%)')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    axes[i].set_ylim(0, max(share_data.max()) * 1.1)

plt.tight_layout()
plt.show()


In [ ]:
## Next Steps: Forecasting Model Development

Based on our analysis, we now have a clear understanding of the MEDIS pharmaceutical sales landscape:

### Key Findings:
1. **Multi-competitor Environment**: 15+ laboratories competing across 4 dosage categories
2. **Market Segmentation**: Clear segmentation by dosage (10mg, 20mg, 40mg, 80mg) and package size
3. **Temporal Patterns**: 7 years of monthly data showing seasonal and growth patterns
4. **Competitive Dynamics**: Market share evolution and competitive responses over time

### Recommended Forecasting Approach:
1. **Hierarchical Time Series**: Model at multiple levels (total → sub-market → package size)
2. **Competitive Features**: Include competitor sales as external regressors
3. **Ensemble Methods**: Combine multiple forecasting approaches for robustness

### Next Development Phases:

#### Phase 1: Baseline Models
- [ ] Simple time series models (ARIMA, Exponential Smoothing)
- [ ] Seasonal decomposition analysis
- [ ] Basic competitive correlation analysis

#### Phase 2: Advanced Models
- [ ] Prophet with competitive regressors
- [ ] XGBoost with engineered features
- [ ] Statistical models with external variables

#### Phase 3: Model Ensemble & Deployment
- [ ] Model combination and validation
- [ ] Streamlit dashboard for forecasting
- [ ] Production-ready pipeline

### Files to Create:
- `forecasting_models.py` - Core forecasting algorithms
- `feature_engineering.py` - Competitive and temporal feature creation
- `model_evaluation.py` - Validation and performance metrics
- `streamlit_dashboard.py` - Interactive forecasting dashboard
